In [2]:
import pandas as pd 
from utils import extract_colors_image,extract_colors_url

In [3]:
url=extract_colors_url("https://nestorevintage.fr/cdn/shop/files/307FDA5D-D3FB-48D6-8DA2-2478DF104BD3.jpg?v=1738348432&width=2048")

c:\Users\BahaARFAOUI\OneDrive - CBTW\Bureau\colorfact\colorfact\utils.py:80: RuntimeWarning: overflow encountered in scalar subtract
  a_std = float(a_opencv - 128)


In [4]:
url.colors

[[58.03921568627452, 251.0, 26.0], [46.27450980392157, 253.0, 23.0]]

In [4]:
image=extract_colors_image(image_path="data/doudoune.png")

In [5]:
image.colors

[[40.3921568627451, -7.0, 6.0], [23.52941176470588, -7.0, 6.0]]

In [6]:
im=image.remove_background_grabcut()

In [20]:
import pandas as pd
data=pd.read_excel("data/final_data.xlsx")

In [21]:
data=data.drop("Unnamed: 0",axis=1)

In [5]:
import pandas as pd
import concurrent.futures
from tqdm import tqdm

def process_image(image_url):
    """Extracts dominant colors from an image URL."""
    try:
        color_extractor = extract_colors_url(image_url)
        return color_extractor.colors
    except Exception as e:
        print(f"Error processing {image_url}: {str(e)}")
        return None

def process_dataset(df: pd.DataFrame, num_workers=8) -> pd.DataFrame:
    """
    Processes a dataset containing image links and adds a 'cielab_colors' column
    with the dominant colors extracted from each image using multithreading.

    Args:
        df (pd.DataFrame): Input dataframe containing a 'Photo produit' column.
        num_workers (int): Number of threads for parallel processing.

    Returns:
        pd.DataFrame: Modified dataframe with 'cielab_colors' column added.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    if 'Photo produit 1' not in df.columns:
        raise ValueError("DataFrame must contain a 'Photo produit' column")

    image_urls = df['Photo produit 1'].tolist()

    # Process images in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(tqdm(executor.map(process_image, image_urls), total=len(image_urls)))

    df['cielab_colors'] = results
    return df

# Example usage:
#df_processed = process_dataset(df_corrected, num_workers=8)


In [14]:
df_processed_2=data.iloc[1150:1350,:]

In [ ]:
df_processed_2_pro = process_dataset(df_processed_2, num_workers=8)


  0%|          | 0/200 [00:00<?, ?it/s]

 36%|███▌      | 72/200 [59:01<4:59:47, 140.53s/it]

Error processing https://images.stockx.com/images/New-Balance-M2002-Protection-Pack-Rain-Cloud-Product.jpg?fit=fill&bg=FFFFFF&w=140&h=75&q=60&dpr=1&trim=color&updated_at=1738193358: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required by KMeans.
Error processing https://images.stockx.com/images/New-Balance-1906R-White-Gold-Product.jpg?fit=fill&bg=FFFFFF&w=140&h=75&q=60&dpr=1&trim=color&updated_at=1738193358: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required by KMeans.


 40%|████      | 80/200 [1:27:02<5:00:44, 150.37s/it] 

In [13]:
df_processed_2_pro.to_excel("data/data_1050_1150.xlsx")

In [3]:
import faiss 
import numpy as np 
import pandas as pd 

In [22]:
data_prev=pd.read_excel("data/data_preview_exploded.xlsx")

In [23]:
data_prev=data_prev.drop("Unnamed: 0",axis=1)

In [6]:
def lab_to_lch(lab):
        L, a, b = lab
        C = np.sqrt(a**2 + b**2)
        H = np.degrees(np.arctan2(b, a)) % 360  # Normalisation correcte
        return [L, C, H]

def lch_to_lab(lch):
    L, C, H = lch
    a = C * np.cos(np.radians(H))
    b = C * np.sin(np.radians(H))
    return [L, a, b]

def generate_harmonic_colors(input_colors_lab):

    input_colors_lch = [lab_to_lch(color) for color in input_colors_lab]
    num_colors = len(input_colors_lab)

    if num_colors == 1:
        L, C, H = input_colors_lch[0]
        triad1 = lch_to_lab([L, C, (H + 120) % 360])
        triad2 = lch_to_lab([L, C, (H + 240) % 360])
        return input_colors_lab + [triad1, triad2]

    elif num_colors == 2:
        chosen_idx = np.random.choice([0, 1])
        L, C, H = input_colors_lch[chosen_idx]
        complementary = lch_to_lab([L, C, (H + 180) % 360])
        return input_colors_lab + [complementary]

    elif num_colors == 3:
        chosen_indices = np.random.choice([0, 1, 2], size=2, replace=False)
        selected_colors = [input_colors_lab[i] for i in chosen_indices]
        return selected_colors

    return input_colors_lab

In [1]:
import yaml
from pyprojroot import here
with open(here("colorfact/configs/outfit.yaml")) as cfg:
    outfit = yaml.load(cfg, Loader=yaml.FullLoader)

In [8]:
data.columns

Index(['Photo produit 1', 'Photo produit 2 ', 'Nom produit ', 'Lien achat',
       'Catégorie produit', 'Genre', 'cielab_colors'],
      dtype='object')

In [115]:
import ast
import numpy as np
import faiss

def recommend_outfit(input_category, data, input_colors, outfit_type, ontology, top_k=3):
    try:
        required_items = ontology["Outfit"][input_category][outfit_type]
    except KeyError as e:
        raise ValueError(f"Invalid category or outfit type: {str(e)}")

    recommendations = {}
    similarity_threshold = 0.2  # 90% similarity

    # Generate harmonized colors for input
    input_colors = generate_harmonic_colors(input_colors)
    query_colors = np.array(input_colors, dtype="float32")
    faiss.normalize_L2(query_colors)

    for item_category in required_items:
        # Filter data for the given category
        filtered_data = data[data["Catégorie produit"] == item_category].copy()

        # Ensure cielab_colors is correctly formatted
        filtered_data["cielab_colors"] = filtered_data["cielab_colors"].apply(ast.literal_eval)
        filtered_data.dropna(inplace=True)
        filtered_data["cielab_colors"] = filtered_data["cielab_colors"].apply(tuple)
        filtered_data.drop_duplicates(inplace=True)

        # Convert to NumPy array
        vectors = np.array([list(t) for t in filtered_data["cielab_colors"]], dtype="float32")
        if vectors.shape[0] == 0:
            continue  # Skip if no data available

        # Normalize and create FAISS index
        faiss.normalize_L2(vectors)
        index = faiss.IndexFlatL2(vectors.shape[1])
        index.add(vectors)

        # Search for closest matches
        distances, indices = index.search(query_colors, top_k)

        seen_ids = set()
        matches = []

        for i in range(indices.shape[0]):
            for j in range(indices.shape[1]):
                idx = indices[i][j]
                if idx >= len(filtered_data) or distances[i][j] > similarity_threshold:
                    continue  # Skip invalid or low-similarity matches

                product_id = filtered_data.iloc[idx]["Photo produit 1"]
                if product_id not in seen_ids:
                    seen_ids.add(product_id)
                    matches.append({
                        "product_id": product_id,
                        "distance": distances[i][j],
                        "metadata": filtered_data.iloc[idx].to_dict(),
                    })
                
                if len(matches) >= top_k:
                    break  # Stop early if enough matches are found

        recommendations[item_category] = matches

    return recommendations


In [111]:
top_k=3
input_colors=[[49.80392156862745, -17.0, -10.0], [41.568627450980394, -16.0, -9.0]]


In [2]:
import json
import unidecode
from rapidfuzz import fuzz, process
from pyprojroot import here
with open(here("colorfact/mapping/product_mapping.json"), "r", encoding="utf-8") as f:
    saved_mapping = json.load(f)
# Load mapping dictionary
mapping = saved_mapping

# Normalize keys in mapping for case-insensitive lookup
normalized_mapping = {unidecode.unidecode(k.lower().strip()): v for k, v in mapping.items()}

# Function to normalize input
def normalize_text(text):
    text = text.lower().strip()  # Lowercase and remove extra spaces
    text = unidecode.unidecode(text)  # Remove accents (é -> e)
    text = text.replace("-", " ").replace("_", " ")  # Replace dashes and underscores with spaces
    return text

# Function to find best match using fuzzy matching
def fuzzy_map(input_text, threshold=80):
    input_text = normalize_text(input_text)

    # 1. Exact match in normalized dictionary
    if input_text in normalized_mapping:
        return normalized_mapping[input_text]

    # 2. Fuzzy match (token-based for better accuracy)
    best_match, score, _ = process.extractOne(input_text, normalized_mapping.keys(), scorer=fuzz.token_sort_ratio)

    if score >= threshold:
        return normalized_mapping[best_match]

    return "No Match"  # No good match found

In [27]:
data_prev['Catégorie produit'] = data_prev['Catégorie produit'].apply(fuzzy_map)

In [116]:
recommend_outfit("T-shirt",data_prev,input_colors,"Casual été",outfit)

{'Jean': [{'product_id': 'https://nestorevintage.fr/cdn/shop/files/IMG_96513.jpg?v=1713712617&width=2048',
   'distance': 0.03934524,
   'metadata': {'Photo produit 1': 'https://nestorevintage.fr/cdn/shop/files/IMG_96513.jpg?v=1713712617&width=2048',
    'Photo produit 2 ': 'https://nestorevintage.fr/cdn/shop/files/IMG_96543.jpg?v=1713712645&width=2048',
    'Nom produit\xa0': 'Jean baggy Bape (XL)',
    'Lien achat': 'https://nestorevintage.fr/products/jean-baggy-bape-6',
    'Catégorie produit': 'Jean',
    'Genre': 'H',
    'cielab_colors': (39.21568627450981, -5.0, -8.0)}},
  {'product_id': 'https://nestorevintage.fr/cdn/shop/files/ABE348CC-E537-4682-A63F-34D8C26706E1.jpg?v=1737289311&width=2048',
   'distance': 0.040083643,
   'metadata': {'Photo produit 1': 'https://nestorevintage.fr/cdn/shop/files/ABE348CC-E537-4682-A63F-34D8C26706E1.jpg?v=1737289311&width=2048',
    'Photo produit 2 ': 'https://nestorevintage.fr/cdn/shop/files/29550E9D-84BC-4F79-9A7A-1C6682C40516.jpg?v=17372893

In [117]:
import ast
import numpy as np
import faiss

def recommend_outfit(input_category, data, input_colors, ontology, top_k=3):
    try:
        outfit_types = ontology["Outfit"][input_category]
    except KeyError as e:
        raise ValueError(f"Invalid category: {str(e)}")

    recommendations = {}
    similarity_threshold = 0.2  # 90% similarity

    # Generate harmonized colors for input
    input_colors = generate_harmonic_colors(input_colors)
    query_colors = np.array(input_colors, dtype="float32")
    faiss.normalize_L2(query_colors)

    for outfit_type, required_items in outfit_types.items():
        outfit_recommendations = {}

        for item_category in required_items:
            # Filter data for the given category
            filtered_data = data[data["Catégorie produit"] == item_category].copy()

            # Ensure cielab_colors is correctly formatted
            filtered_data["cielab_colors"] = filtered_data["cielab_colors"].apply(ast.literal_eval)
            filtered_data.dropna(inplace=True)
            filtered_data["cielab_colors"] = filtered_data["cielab_colors"].apply(tuple)
            filtered_data.drop_duplicates(inplace=True)

            # Convert to NumPy array
            vectors = np.array([list(t) for t in filtered_data["cielab_colors"]], dtype="float32")
            if vectors.shape[0] == 0:
                continue  # Skip if no data available

            # Normalize and create FAISS index
            faiss.normalize_L2(vectors)
            index = faiss.IndexFlatL2(vectors.shape[1])
            index.add(vectors)

            # Search for closest matches
            distances, indices = index.search(query_colors, top_k)

            seen_ids = set()
            matches = []

            for i in range(indices.shape[0]):
                for j in range(indices.shape[1]):
                    idx = indices[i][j]
                    if idx >= len(filtered_data) or distances[i][j] > similarity_threshold:
                        continue  # Skip invalid or low-similarity matches

                    product_id = filtered_data.iloc[idx]["Photo produit 1"]
                    if product_id not in seen_ids:
                        seen_ids.add(product_id)
                        matches.append({
                            "product_id": product_id,
                            "distance": distances[i][j],
                            "metadata": filtered_data.iloc[idx].to_dict(),
                        })
                    
                    if len(matches) >= top_k:
                        break  # Stop early if enough matches are found

            outfit_recommendations[item_category] = matches

        recommendations[outfit_type] = outfit_recommendations

    return recommendations


In [119]:
rec=recommend_outfit("T-shirt", data_prev, input_colors, outfit)


In [121]:
rec.keys()

dict_keys(['Casual été', 'Casual hiver', 'Sportswear', 'Professionnel'])

In [124]:
rec["Casual été"]["Sneakers"]

[{'product_id': 'https://fr.sandro-paris.com/dw/image/v2/BCMW_PRD/on/demandware.static/-/Sites-master-catalog/default/dwe4d5ac03/images/packshot/Sandro_SFACH01164-47_F_P.jpg?sw=650&sh=650',
  'distance': 0.11775888,
  'metadata': {'Photo produit 1': 'https://fr.sandro-paris.com/dw/image/v2/BCMW_PRD/on/demandware.static/-/Sites-master-catalog/default/dwe4d5ac03/images/packshot/Sandro_SFACH01164-47_F_P.jpg?sw=650&sh=650',
   'Photo produit 2 ': '-',
   'Nom produit\xa0': 'Baskets Flame',
   'Lien achat': 'https://fr.sandro-paris.com/fr/p/baskets-flame/SFACH01164_47.html',
   'Catégorie produit': 'Sneakers',
   'Genre': 'F',
   'cielab_colors': (50.588235294117645, 1.0, -8.0)}},
 {'product_id': 'https://fr.sandro-paris.com/dw/image/v2/BCMW_PRD/on/demandware.static/-/Sites-master-catalog/default/dw5279f87f/images/packshot/Sandro_SFACH00860-20_V_P.jpg?sw=650&sh=650',
  'distance': 0.15400456,
  'metadata': {'Photo produit 1': 'https://fr.sandro-paris.com/dw/image/v2/BCMW_PRD/on/demandware.s

In [3]:
pathh="data/imagesremoved.png"

In [4]:
from utils import get_category,matching_products
import pandas as pd 
from utils import extract_colors_image,extract_colors_url

In [5]:
colors=extract_colors_image(image_path=pathh)

In [6]:
colors.colors

[[77.25490196078432, 4.0, 14.0], [62.35294117647059, 7.0, 16.0]]

In [7]:
cat=get_category(pathh)

In [8]:
genre=cat.get_category_gender()

In [9]:
genre

{'genre': 'H', 'category': 'Blouson'}

In [10]:
data=pd.read_excel("data/data_preview_exploded.xlsx")

In [11]:
data=data.drop("Unnamed: 0",axis=1)

In [12]:
data['Genre']=data["Genre"].apply(lambda x : x.strip())

In [13]:
data['Catégorie produit'] = data['Catégorie produit'].apply(fuzzy_map)

In [14]:
match=matching_products(data=data,ontologie=outfit)

In [15]:
match.recommend_outfit(genre.get("category"),colors.colors,genre.get("genre"))

{'Casual 1': {'Jean': [{'product_id': 'https://ntmb.it/cdn/shop/products/jeans-ultra-baggy-white-patchwork-215632.jpg?v=1694471136&width=3000',
    'distance': 0.00083956117,
    'metadata': {'Photo produit 1': 'https://ntmb.it/cdn/shop/products/jeans-ultra-baggy-white-patchwork-215632.jpg?v=1694471136&width=3000',
     'Photo produit 2 ': '-',
     'Nom produit\xa0': 'Jeans ULTRA Baggy Patchwork',
     'Lien achat': 'https://ntmb.it/collections/jeans/products/jeans-ultra-baggy-white-patchwork',
     'Catégorie produit': 'Jean',
     'Genre': 'H',
     'cielab_colors': (72.54901960784314, 4.0, 11.0)}},
   {'product_id': 'https://ntmb.it/cdn/shop/products/jeans-straight-baggy-cream-flowers-embroidery-653720.jpg?v=1694470562&width=3000',
    'distance': 0.0021518648,
    'metadata': {'Photo produit 1': 'https://ntmb.it/cdn/shop/products/jeans-straight-baggy-cream-flowers-embroidery-653720.jpg?v=1694470562&width=3000',
     'Photo produit 2 ': '-',
     'Nom produit\xa0': 'Jeans Straight 

In [16]:
urls=data["Photo produit 1"].tolist()

In [18]:
len(urls)

3306

In [19]:
import random

random_selection = random.sample(urls, 20)

In [20]:
import os
import requests
from io import BytesIO
from PIL import Image, UnidentifiedImageError

def download_image(image_url,save_folder):
    """
    Downloads an image from a URL and saves it to the specified local folder.
    """
    try:
        headers = {"User-Agent": "Mozilla/5.0"}  # Mimic a browser request
        response = requests.get(image_url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error if request fails

        try:
            image = Image.open(BytesIO(response.content))

            # Ensure the folder exists
            os.makedirs(save_folder, exist_ok=True)

            # Generate a filename from the URL
            filename = os.path.join(save_folder, os.path.basename(image_url.split("?")[0]))

            # Save the image
            image.save(filename)
            print(f"Image saved to {filename}")
            return filename

        except UnidentifiedImageError:
            print(f"Error: Cannot identify image from {image_url}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None


In [22]:
for item in random_selection: 
    download_image(item,"data/images/")

Image saved to data/images/D79524CE-425D-4A39-868E-DEBFF208A823.jpg
Image saved to data/images/EEBB3527-9E6D-4D17-B8C7-CE290E132BEC.jpg
Image saved to data/images/CS1015_Male_OrganicOversizedHood-OceanGreen_2.jpg
Image saved to data/images/hoodie-zip-crown-pitcher-736902_2048x2048.jpg
Image saved to data/images/pull-mind-868755.jpg
Image saved to data/images/CS1015_Male_OrganicOversizedHood-UltraViolet_2.jpg
Image saved to data/images/CS2052_Female_WomenClassicOrganicCrew-SteelBlue_2.jpg
Image saved to data/images/TshirtRockyVSunyzebreSS23-1_2000x.jpg
Image saved to data/images/CS5091_Female_MerinoQuarterZip-DeepBlack_2.jpg
Image saved to data/images/D8FEC524-E82B-4443-9B7C-F799DB76DA97.jpg
Image saved to data/images/D4665C20-E47F-4BEA-92C4-E0AC544257E9.jpg
Image saved to data/images/denim-bucket-hats-fragment-679513.jpg
Image saved to data/images/CS5087_Female_WomenClassicMerinoWoolCrew-DustyOlive_2.jpg
Image saved to data/images/jeans-straight-baggy-flame-patchwork-684700.jpg
Image s